In [1]:
%load_ext Cython

In [2]:
import sys
sys.path[0] = '/home/bram/ANTS/'

In [27]:
%%cython -a
from cythonic.plugins.positions cimport point, ant_state, full_state
from cythonic.plugins.functions cimport transform
from cythonic.plugins.sens_functions cimport lin
from cythonic.plugins.drop_functions cimport exp_decay
import numpy as np
from libc.math cimport M_PI as PI
cimport cython
from cythonic.plugins.rng cimport RNG

@cython.cdivision(True)
@cython.wraparound(False)
@cython.boundscheck(False)
@cython.nonecheck(False)
cdef class Ant:
    cdef:
        #physical properties
        readonly double l
        readonly unsigned int id
        readonly point limits

        # environment interaction
        readonly double gain,sens_offset

        #states
        full_state state
#         bint foodbound
#         bint out_of_bounds
#         double _azimuth
#         point _pos
#         point _left
#         point _right
#         bint active
#         double v
#         double time #time since last visit of nest/food
#         double rng_time #timer for the rng

        # drop quantity related
        readonly double[2] q_observed
        readonly double _drop_quantity, return_factor,  drop_beta
        readonly str drop_fun

        # sensing related
        readonly RNG rng
    " default agent acting in a system "
    def __cinit__(self, double l, unsigned int id,
                    double speed, double gain, double sens_offset,
                    double[:] limits, double q, double return_factor,
                    str drop_fun, double drop_beta, double rng_gamma):
        " (virtual) hardware characteristics "
        self.id = id
        self.l = l
        # self.v = speed
        self.gain = gain
        self.sens_offset = sens_offset

        " environment "
        self.q_observed[0]=0
        self.q_observed[1]=0

        " dropping related "
        self.drop_beta = drop_beta
        self.drop_fun = drop_fun
        self._drop_quantity = q
        self.return_factor = return_factor

        " private RNG "
        self.rng = RNG(rng_gamma)

        " state "
        # self.foodbound = False
        # self.out_of_bounds = False
        # self.time = 0.
        # self.active = False

        " constraints "
        self.limits = point(limits[0], limits[1])

    @cython.wraparound(False)
    @cython.boundscheck(False)
    @cython.nonecheck(False)
    cdef void point_state(self, full_state *S):
        " point ant states to members of struct full_state "
        self.state = S[0]
#         self.foodbound = S[0].foodbound
#         self.out_of_bounds = S[0].out_of_bounds
#         self.active = S[0].active
#         self._azimuth = S[0]._azimuth
#         self.v = S[0].v
#         self._pos = S[0]._pos
#         self._left = S[0]._left
#         self._right = S[0]._right
#         self.rng_time = S[0].rng_time
#         self.time = S[0].time

#     cdef void observe(self,str observe_fun, double[:] Q):
#         " observe pheromone Q[0] (left) and Q[1] (right) "
#         if observe_fun =='linear':
#             self.q_observed[0] = lin(&Q[0])
#             self.q_observed[1] = lin(&Q[1])

#     cdef double return_drop_quantity(self, double *dt):
#         """ return the pheromone to be dropped on the map,
#             possibly based on internal timer self.time """
#         cdef double Q=0 # drop quantity
#         if self.drop_fun == 'constant':
#             Q = self._drop_quantity
#         elif self.drop_fun == 'exp_decay':
#             Q = exp_decay(&self._drop_quantity, &self.time,&self.drop_beta)
#         if self.foodbound:
#             return Q*dt[0]
#         else:
#             " possibly drop more pheromone when food is found "
#             return self.return_factor*Q*dt[0]

#     cdef public void rotate(self, double * dt):
#         " rotate the ant "
#         cdef double d_teta = dt[0]*self.gain*180./PI*(
#             self.q_observed[0]-self.q_observed[1])
#         """STILL NO NOISE!!! """
#         self.increase_azimuth(&d_teta)

    cdef bint correct_bounds(self):
        " Check limits, place ant at boundary of neccesary "
        cdef bint xbound = True # assume out of bounds
        cdef bint ybound = True

        " slightly (overly) complicated code to ensure all-C performance "
        if self.state._pos.x >= self.limits.x: self.state._pos.x = self.limits.x #upper limit x
        elif self.state._pos.x <= 0.: self.state._pos.x = 0. #lower limit x
        else: xbound = False
        if self.state._pos.y >= self.limits.y: self.state._pos.y = self.limits.y #upper limit y
        elif self.state._pos.y <= 0: self.state._pos.y = 0. # lower limit y
        else: ybound = False #turns out, not out of bounds

        " return boolean for out_of_bounds"
        if xbound or ybound:
            return True
        else:
            return False
        
#     cdef full_state get_state(self):
#         " wrap the ant state in a struct "
#         cdef full_state s
#         s.foodbound = self.foodbound
#         s.out_of_bounds = self.out_of_bounds
#         s.active = self.active
#         s._azimuth = self._azimuth
#         s.v = self.v
#         s._pos = self._pos
#         s._left = self._left
#         s._right = self._right
#         s.rng_time = self.rng_time
#         s.time = self.time
#         return s

#     cdef readonly full_state gradient_step(self,double dt, str observe_fun, double[:] Q):
#         " do the sequence of differential stepping "
#         " 1: sniff pheromone, 2: rotate, 3: step, 4: update sensors "
#         self.observe(observe_fun, Q)
#         self.rotate(&dt)
#         self.step(&dt)
#         self.set_sensors()
#         return self.get_state()

#     cdef readonly void step(self,double * dt):
#         """ do a step in the current direction, do boundary checking as well """
#         # self.rng.add_t(dt) #update timer of the RNG
#         self.time += dt[0] #update internal timer

#         " Update position "
#         cdef double dL = self.v*dt[0]
#         self._pos = transform(self._azimuth,&dL,&self._pos)

#         " correct for boundary "
#         self.out_of_bounds = self.correct_bounds()

#     cdef void set_sensors(self):
#         " update the position of the sensors "
#         self._left = transform(self._azimuth+self.sens_offset,&self.l,&self._pos)
#         self._right = transform(self._azimuth-self.sens_offset,&self.l,&self._pos)

#     # cdef void init_positions(self, double x, double y):
#     #     " call when initializing the ant, setdouble[:] xy the position "
#     #     self._pos = point(x,y)
#     #     self.set_sensors()

#     cdef void activate(self,ant_state x):
#         " activate the ant and set the state "
#         self._pos = point(x.x, x.y)
#         self._azimuth = x.theta
#         self.set_sensors()
#         self.active = True


#     cdef void increase_azimuth(self, double * d_teta):
#         " C-implementation of property azimuth.setter "
#         if d_teta[0]+ self._azimuth >= 360:
#             self._azimuth += d_teta[0]-360.
#         elif d_teta[0] + self._azimuth < 0:
#             self._azimuth += d_teta[0] + 360.
#         else:
#             self._azimuth += d_teta[0]

#     " some legacy properties "
#     @property
#     def pos(self):
#         return (self._pos.x, self._pos.y)
#     @property
#     def left(self):
#         return (self._left.x, self._left.y)
#     @property
#     def right(self):
#         return (self._right.x, self._right.y)
#     @property
#     def azimuth(self):
#         return self._azimuth
#     @azimuth.setter
#     def azimuth(self,double x):
#         if x >= 360.0:
#             self._azimuth = x-360.0
#         else:
#             self._azimuth = x

#     def __enter__(self):
#         """ return when class is casted in 'with Ant as ..:"""
#         return self
#     def __exit__(self, type, value, traceback):
#         """ Accompanies __enter__"""
#         pass



In [113]:
%%cython -a
from cythonic.plugins.positions cimport point
from libc.math cimport sin as csin,cos as ccos
cimport numpy as np
import numpy as np

DEF PI = 3.1415926535897932384626433832795028841971
cdef double deg(double *rad):
    return rad[0]*180/PI

cdef double rad(double * deg):
    return deg[0]*PI/180

cdef double d,r
r = 2*PI
d = deg(&r)
print(r)
print(d)
cdef point p = point(2,2)
cdef double [2] offset = np.array([10.,10.],dtype=np.float_)

cdef point transform(double *teta_deg, double *arm, double[2]* xy):
    " return location of a point based on: [x;y] = T*[arm;0]+offset"
    cdef double teta = rad(teta_deg)
    cdef point result = point(xy[0][0]+ccos(teta)*arm[0],xy[0][1]+csin(teta)*arm[0])
    return result


cdef point P
P = transform(&d,&d,&p.xy)

6.283185307179586
360.0


In [33]:
%%cython -a
cdef class X:
    cdef double x
    def new(self, double x):
        self.x = x
        
cdef class Y(X):
    cdef double y
    def __cinit__(self,double x, double y):
        self.new(x)
        self.y =y
        print(self.y)

cdef Y cY =  Y(1.,2.)

2.0
